# Kelly Criterion

### **Problem Statement - How much fraction of our Wealth should we bet?**

Winning probability = p

Losing probability = q

If win, fraction of bet gained = g

If lose, fraction of bet lost = l


### Idea: 

If win: 
$$S = S + S*f*g = (1+fg)S$$

If lose:
$$S = S - S*f*l = (1+fl)S$$

Upon playing n games, we have growth rate of 
$$r = (1+fg)^{np}(1+fl)^{nq}$$


We want to maximize r, thus we can find the local maxima, ie. 
$$\frac{dlnr}{df} = \frac{npg}{1+fg} + \frac{nql}{1+fl} = 0$$
$$ (1-fl)(pg) - (1+fg)(ql) = 0 $$
$$ pg - flpg - ql - fgql = 0$$
$$ flg(p+q) = pg - ql$$
$$ f = \frac{p}{l} + \frac{q}{g} $$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sim = 1000      # Number of simulations
n = 50          # Bets per simulation
p = 0.3         # Win probability
q = 1-p         # Loss probability
g = 5           # Win multiplier (e.g., bet $1 → win $10)
l = 1           # Loss multiplier (bet $1 → lose $1)
b = g / l       # Odds (win/loss ratio)
kelly_fraction = p/l - q/g

initial_wealth = 1e7  
experiments = np.zeros((sim, n+1)) 
experiments[:, 0] = initial_wealth

np.random.seed(42)
for i in range(sim):
    wealth = initial_wealth
    for j in range(1, n+1): 
        if np.random.random() < p:
            # Win: wealth increases by kelly_fraction * g
            wealth += wealth * kelly_fraction * g
        else:
            # Loss: wealth decreases by kelly_fraction * l  
            wealth -= wealth * kelly_fraction * l
        
        wealth = max(wealth, 0)
        experiments[i, j] = wealth

In [21]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np

# Assuming 'experiments' is your simulated data
# If not, recreate it with the corrected Kelly simulation:
sim = 50
n = 100
experiments = np.zeros((sim, n+1))
initial_wealth = 1e7

# Generate sample data (replace with your actual experiments array)
np.random.seed(42)
for i in range(sim):
    wealth = initial_wealth
    experiments[i, 0] = wealth
    for j in range(1, n+1):
        # Simplified random walk for demonstration
        wealth = wealth * (1 + np.random.normal(0.001, 0.02))
        experiments[i, j] = wealth

# Create interactive Plotly figure
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Wealth Trajectories', 'Distribution of Final Wealth', 
                    'Growth Rate Distribution', 'Cumulative Statistics'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": True}, {"secondary_y": False}]],
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# ========== Plot 1: Wealth Trajectories (Main Chart) ==========
for i in range(min(50, len(experiments))):
    fig.add_trace(
        go.Scatter(
            x=list(range(len(experiments[i]))),
            y=experiments[i],
            mode='lines',
            line=dict(width=1, color='rgba(100, 149, 237, 0.4)'),  # Cornflower blue
            name=f'Simulation {i+1}',
            showlegend=False,
            hovertemplate='Bet: %{x}<br>Wealth: $%{y:,.0f}<br>Return: %{customdata:.1%}<extra></extra>',
            customdata=[experiments[i][j]/initial_wealth - 1 for j in range(len(experiments[i]))]
        ),
        row=1, col=1
    )

# Add mean trajectory
mean_trajectory = experiments.mean(axis=0)
fig.add_trace(
    go.Scatter(
        x=list(range(len(mean_trajectory))),
        y=mean_trajectory,
        mode='lines',
        line=dict(width=3, color='#FF6B6B'),  # Coral red
        name='Average',
        hovertemplate='Bet: %{x}<br>Avg Wealth: $%{y:,.0f}<extra></extra>'
    ),
    row=1, col=1
)

# Add median trajectory
median_trajectory = np.median(experiments, axis=0)
fig.add_trace(
    go.Scatter(
        x=list(range(len(median_trajectory))),
        y=median_trajectory,
        mode='lines',
        line=dict(width=3, color='#4ECDC4', dash='dash'),  # Turquoise
        name='Median',
        hovertemplate='Bet: %{x}<br>Median Wealth: $%{y:,.0f}<extra></extra>'
    ),
    row=1, col=1
)

# ========== Plot 2: Distribution of Final Wealth ==========
final_wealths = experiments[:, -1]
fig.add_trace(
    go.Histogram(
        x=final_wealths,
        nbinsx=30,
        name='Final Wealth',
        marker_color='#95E1D3',  # Light teal
        opacity=0.8,
        hovertemplate='Wealth: $%{x:,.0f}<br>Count: %{y}<extra></extra>'
    ),
    row=1, col=2
)

# Add vertical lines for statistics
stats = {
    'Mean': np.mean(final_wealths),
    'Median': np.median(final_wealths),
    '10th Pctl': np.percentile(final_wealths, 10),
    '90th Pctl': np.percentile(final_wealths, 90)
}

colors = {'Mean': '#FF6B6B', 'Median': '#4ECDC4', '10th Pctl': '#FFD166', '90th Pctl': '#118AB2'}
for stat_name, stat_value in stats.items():
    fig.add_vline(
        x=stat_value,
        line_dash="dash",
        line_color=colors[stat_name],
        opacity=0.7,
        row=1, col=2
    )
    
    # Add annotation
    fig.add_annotation(
        x=stat_value,
        y=0.9,
        yref="paper",
        text=f"{stat_name}: ${stat_value:,.0f}",
        showarrow=False,
        font=dict(size=10, color=colors[stat_name]),
        row=1, col=2
    )

# ========== Plot 3: Growth Rate Distribution ==========
growth_rates = np.diff(np.log(experiments), axis=1).flatten()
fig.add_trace(
    go.Histogram(
        x=growth_rates,
        nbinsx=40,
        name='Growth Rates',
        marker_color='#FFD166',  # Yellow
        opacity=0.8,
        hovertemplate='Growth: %{x:.3%}<br>Density: %{y}<extra></extra>'
    ),
    row=2, col=1
)

# Add KDE curve
from scipy import stats
kde = stats.gaussian_kde(growth_rates)
x_kde = np.linspace(growth_rates.min(), growth_rates.max(), 200)
y_kde = kde(x_kde)

fig.add_trace(
    go.Scatter(
        x=x_kde,
        y=y_kde,
        mode='lines',
        line=dict(width=2, color='#E76F51'),  # Terracotta
        name='KDE',
        yaxis="y2",
        hovertemplate='Growth: %{x:.3%}<br>Density: %{y:.3f}<extra></extra>'
    ),
    row=2, col=1
)

# ========== Plot 4: Cumulative Statistics ==========
# Calculate cumulative statistics
bet_numbers = list(range(experiments.shape[1]))
cumulative_mean = experiments.mean(axis=0)
cumulative_std = experiments.std(axis=0)
cumulative_min = experiments.min(axis=0)
cumulative_max = experiments.max(axis=0)

# Fill between mean ± std
fig.add_trace(
    go.Scatter(
        x=bet_numbers + bet_numbers[::-1],
        y=list(cumulative_mean + cumulative_std) + list(cumulative_mean - cumulative_std)[::-1],
        fill='toself',
        fillcolor='rgba(100, 149, 237, 0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        name='Mean ± 1σ',
        showlegend=True,
        hovertemplate='Bet: %{x}<br>Wealth Range<br>extra'
    ),
    row=2, col=2
)

# Add mean line
fig.add_trace(
    go.Scatter(
        x=bet_numbers,
        y=cumulative_mean,
        mode='lines',
        line=dict(width=2, color='#264653'),  # Dark blue-green
        name='Mean',
        hovertemplate='Bet: %{x}<br>Mean: $%{y:,.0f}<extra></extra>'
    ),
    row=2, col=2
)

# Add min/max bounds
fig.add_trace(
    go.Scatter(
        x=bet_numbers,
        y=cumulative_max,
        mode='lines',
        line=dict(width=1, color='rgba(230, 57, 70, 0.5)', dash='dot'),
        name='Maximum',
        hovertemplate='Bet: %{x}<br>Max: $%{y:,.0f}<extra></extra>'
    ),
    row=2, col=2
)

fig.add_trace(
    go.Scatter(
        x=bet_numbers,
        y=cumulative_min,
        mode='lines',
        line=dict(width=1, color='rgba(57, 230, 100, 0.5)', dash='dot'),
        name='Minimum',
        hovertemplate='Bet: %{x}<br>Min: $%{y:,.0f}<extra></extra>'
    ),
    row=2, col=2
)

# ========== Update Layout and Axes ==========
fig.update_layout(
    # title=dict(
    #     text='<b>Kelly Criterion Simulation Analysis</b><br><span style="font-size:14px">Wealth Evolution Across 50 Simulations</span>',
    #     x=0.5,
    #     xanchor='center',
    #     font=dict(size=20, family='Arial, sans-serif')
    # ),
    template='plotly_white',
    height=900,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    hovermode='x unified',
    plot_bgcolor='rgba(245, 247, 250, 0.8)',
    paper_bgcolor='white'
)

# Update axes
fig.update_xaxes(title_text="Bet Number", row=1, col=1)
fig.update_yaxes(title_text="Wealth ($)", tickprefix="$", row=1, col=1)
fig.update_xaxes(title_text="Final Wealth ($)", tickprefix="$", row=1, col=2)
fig.update_yaxes(title_text="Count", row=1, col=2)
fig.update_xaxes(title_text="Growth Rate", tickformat=".1%", row=2, col=1)
fig.update_yaxes(title_text="Count", row=2, col=1)
fig.update_yaxes(title_text="Density", row=2, col=1, secondary_y=True)
fig.update_xaxes(title_text="Bet Number", row=2, col=2)
fig.update_yaxes(title_text="Wealth ($)", tickprefix="$", row=2, col=2)

# Add annotations with summary statistics
summary_text = f"""
<b>Summary Statistics</b><br>
Initial Wealth: ${initial_wealth:,.0f}<br>
Simulations: {sim}<br>
Bets per Simulation: {n}<br>
Mean Final Wealth: ${np.mean(final_wealths):,.0f}<br>
Median Final Wealth: ${np.median(final_wealths):,.0f}<br>
Volatility (Std): ${np.std(final_wealths):,.0f}<br>
Max Drawdown: {((experiments.min(axis=1) / experiments.max(axis=1) - 1).mean()):.1%}
"""

fig.add_annotation(
    text=summary_text,
    xref="paper", yref="paper",
    x=0.02, y=0.98,
    showarrow=False,
    align="left",
    bgcolor="rgba(255, 255, 255, 0.8)",
    bordercolor="black",
    borderwidth=1,
    font=dict(size=11, family="Courier New, monospace")
)

# Add interactive buttons
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            x=0.5,
            y=1.15,
            xanchor="center",
            showactive=True,
            buttons=list([
                dict(
                    label="Linear Scale",
                    method="update",
                    args=[{"visible": [True]*len(fig.data)}, 
                          {"yaxis.type": "linear", "yaxis2.type": "linear",
                           "yaxis3.type": "linear", "yaxis4.type": "linear"}]
                ),
                dict(
                    label="Log Scale",
                    method="update",
                    args=[{"visible": [True]*len(fig.data)}, 
                          {"yaxis.type": "log", "yaxis2.type": "log",
                           "yaxis3.type": "linear", "yaxis4.type": "log"}]
                )
            ]),
        )
    ]
)

# Show the figure
fig.show()

# Kelly and Sharpe ratio

In [108]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)
import numpy as np

def simulate_kelly_vectorized_rebalancing(mu, vol, rf, frac, sim=100000, n=252):
    """
    Fully vectorized Kelly simulation with rebalancing
    """
    dt = 1.0 / n
    
    # Generate ALL random returns at once
    risky_returns = mu * dt + vol * np.sqrt(dt) * np.random.randn(sim, n)
    safe_returns = rf * dt  # Scalar, same for all
    
    # Growth factor each period
    growth_factors = 1 + frac * risky_returns + (1 - frac) * safe_returns
    
    # Cumulative product along time axis (axis=1)
    cumulative_growth = np.cumprod(growth_factors, axis=1)
    
    # Add initial wealth column
    wealth = np.ones((sim, n + 1))
    wealth[:, 1:] = cumulative_growth
    
    return wealth

In [109]:
from joblib import Parallel, delayed

def parallel_kelly_joblib(mu,vol,rf):
    
    fractions = np.linspace(0, 1.5, 31)
    
    # Use joblib's simple syntax
    results = Parallel(n_jobs=-1, verbose=10)(
        delayed(lambda f: {
            "frac": f,
            "mean_log": np.mean(np.log(
                simulate_kelly_vectorized_rebalancing(mu, vol, rf, f, 100000, 252)[:, -1]
            ))
        })(frac) for frac in fractions
    )
    
    optimal_idx = np.argmax([x["mean_log"] for x in results])
    optimal_frac = results[optimal_idx]["frac"]
    
    return results, optimal_frac

In [110]:
mu, vol, rf = 0.05, 0.3, 0.03
wealth, optimal = parallel_kelly_joblib(mu,vol,rf)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   32.6s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   46.1s
[Parallel(n_jobs=-1)]: Done  20 out of  31 | elapsed:   50.2s remaining:   27.6s
[Parallel(n_jobs=-1)]: Done  24 out of  31 | elapsed:   55.7s remaining:   16.2s
[Parallel(n_jobs=-1)]: Done  28 out of  31 | elapsed:  1.0min remaining:    6.6s
[Parallel(n_jobs=-1)]: Done  31 out of  31 | elapsed:  1.0min finished


In [115]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import numpy as np
import pandas as pd
from scipy import stats

# ============================================================================
# 1. VECTORIZED KELLY SIMULATION WITH REBALANCING
# ============================================================================

def simulate_kelly_vectorized(mu, vol, rf, frac, sim=10000, n=252):
    """Fully vectorized Kelly simulation with rebalancing"""
    dt = 1.0 / n
    
    # Generate all random shocks
    z = np.random.randn(sim, n)
    
    # Portfolio parameters (with rebalancing)
    portfolio_mu = frac * mu + (1 - frac) * rf
    portfolio_vol = frac * vol
    
    # Log-space calculation (mathematically correct for GBM)
    log_drift = (portfolio_mu - 0.5 * portfolio_vol**2) * dt
    log_shocks = portfolio_vol * np.sqrt(dt) * z
    
    # Cumulative log returns
    cumulative_log = np.cumsum(log_drift + log_shocks, axis=1)
    
    # Wealth paths
    wealth = np.ones((sim, n + 1))
    wealth[:, 1:] = np.exp(cumulative_log)
    
    return wealth

# ============================================================================
# 2. GENERATE DATA FOR VISUALIZATION
# ============================================================================

# Parameters
mu, vol, rf = 0.05, 0.30, 0.03
kelly_fraction = (mu - rf) / (vol**2)  # Should be ~0.222

# Test different fractions
fractions = np.linspace(0, 1.0, 41)  # 0% to 100%
sim_per_fraction = 10000
n_periods = 252

print(f"Kelly optimal fraction: {kelly_fraction:.3f}")
print(f"Testing {len(fractions)} fractions with {sim_per_fraction:,} simulations each")

# Store results
results = []
wealth_samples = {}  # Store sample paths for key fractions

for frac in fractions:
    # Run simulation
    wealth_paths = simulate_kelly_vectorized(mu, vol, rf, frac, 
                                             sim=sim_per_fraction, 
                                             n=n_periods)
    
    final_wealth = wealth_paths[:, -1]
    
    # Compute statistics
    log_growth = np.mean(np.log(final_wealth))
    mean_wealth = np.mean(final_wealth)
    median_wealth = np.median(final_wealth)
    std_wealth = np.std(final_wealth)
    sharpe = (mean_wealth - 1) / std_wealth  # Simple Sharpe (assuming initial wealth=1)
    
    # Store fraction results
    results.append({
        'fraction': frac,
        'log_growth': log_growth,
        'mean_wealth': mean_wealth,
        'median_wealth': median_wealth,
        'std_wealth': std_wealth,
        'sharpe': sharpe,
        'prob_ruin': np.mean(final_wealth < 0.5),  # Probability of losing >50%
        'final_wealth_dist': final_wealth
    })
    
    # Store sample paths for key fractions
    if frac in [0, kelly_fraction, 0.5, 1.0]:
        wealth_samples[frac] = wealth_paths[:50]  # First 50 paths for visualization

# Convert to DataFrame
df_results = pd.DataFrame(results)

# Find optimal fractions
optimal_log_idx = df_results['log_growth'].idxmax()
optimal_log_frac = df_results.loc[optimal_log_idx, 'fraction']
optimal_mean_idx = df_results['mean_wealth'].idxmax()
optimal_mean_frac = df_results.loc[optimal_mean_idx, 'fraction']

print(f"\nMonte Carlo Results:")
print(f"Optimal for log growth: {optimal_log_frac:.3f} (Kelly formula: {kelly_fraction:.3f})")
print(f"Optimal for mean wealth: {optimal_mean_frac:.3f}")

# ============================================================================
# 3. CREATE INTERACTIVE PLOTLY DASHBOARD
# ============================================================================

# Create subplot figure
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        'Wealth Evolution for Different Fractions',
        'Expected Log Growth vs Fraction',
        'Distribution of Final Wealth',
        'Key Statistics vs Fraction',
        'Probability of Significant Loss',
        'Sample Wealth Paths Comparison'
    ),
    specs=[
        [{"type": "scatter", "colspan": 2}, None],
        [{"type": "scatter"}, {"type": "histogram"}],
        [{"type": "scatter"}, {"type": "scatter"}]
    ],
    vertical_spacing=0.1,
    horizontal_spacing=0.15
)

# ========== Plot 1: Wealth Evolution ==========
# Show sample paths for key fractions
key_fractions = sorted(wealth_samples.keys())
colors = px.colors.qualitative.Set1

for idx, frac in enumerate(key_fractions):
    wealth_paths = wealth_samples[frac]
    
    # Plot each path
    for i in range(min(10, len(wealth_paths))):
        fig.add_trace(
            go.Scatter(
                x=list(range(len(wealth_paths[i]))),
                y=wealth_paths[i],
                mode='lines',
                line=dict(width=1, color=colors[idx % len(colors)]),
                opacity=0.3,
                name=f'{frac:.2f} fraction',
                showlegend=False,
                hovertemplate='Day: %{x}<br>Wealth: %{y:.2f}<extra></extra>'
            ),
            row=1, col=1
        )
    
    # Add mean path
    mean_path = wealth_paths.mean(axis=0)
    fig.add_trace(
        go.Scatter(
            x=list(range(len(mean_path))),
            y=mean_path,
            mode='lines',
            line=dict(width=3, color=colors[idx % len(colors)]),
            name=f'Mean (f={frac:.2f})',
            hovertemplate='Day: %{x}<br>Mean Wealth: %{y:.2f}<extra></extra>'
        ),
        row=1, col=1
    )

# ========== Plot 2: Log Growth vs Fraction ==========
fig.add_trace(
    go.Scatter(
        x=df_results['fraction'],
        y=df_results['log_growth'],
        mode='lines+markers',
        line=dict(width=3, color='#2E86AB'),
        marker=dict(size=8),
        name='Expected Log Growth',
        hovertemplate='Fraction: %{x:.3f}<br>Log Growth: %{y:.4f}<extra></extra>'
    ),
    row=2, col=1
)

# Add Kelly optimal line
fig.add_vline(
    x=kelly_fraction,
    line_dash="dash",
    line_color="#FF6B6B",
    opacity=0.8,
    row=2, col=1
)

fig.add_annotation(
    x=kelly_fraction,
    y=0.95,
    yref="paper",
    text=f"Kelly Optimal: {kelly_fraction:.3f}",
    showarrow=True,
    arrowhead=2,
    arrowcolor="#FF6B6B",
    row=2, col=1
)

# Add Monte Carlo optimal point
fig.add_trace(
    go.Scatter(
        x=[optimal_log_frac],
        y=[df_results.loc[optimal_log_idx, 'log_growth']],
        mode='markers',
        marker=dict(size=15, color='#FF6B6B', symbol='star'),
        name='MC Optimal',
        hovertemplate='MC Optimal: %{x:.3f}<br>Log Growth: %{y:.4f}<extra></extra>'
    ),
    row=2, col=1
)

# ========== Plot 3: Distribution of Final Wealth ==========
# Show distributions for key fractions
for idx, frac in enumerate(key_fractions):
    final_wealth = df_results.loc[df_results['fraction'] == frac, 'final_wealth_dist'].iloc[0]
    
    fig.add_trace(
        go.Histogram(
            x=final_wealth,
            nbinsx=50,
            name=f'f={frac:.2f}',
            opacity=0.6,
            marker_color=colors[idx % len(colors)],
            histnorm='probability density',
            showlegend=True,
            hovertemplate='Wealth: %{x:.2f}<br>Density: %{y:.3f}<extra></extra>'
        ),
        row=2, col=2
    )

# Add KDE curves
x_range = np.linspace(0, 5, 200)
for idx, frac in enumerate(key_fractions):
    final_wealth = df_results.loc[df_results['fraction'] == frac, 'final_wealth_dist'].iloc[0]
    
    # Fit KDE
    kde = stats.gaussian_kde(final_wealth)
    y_kde = kde(x_range)
    
    fig.add_trace(
        go.Scatter(
            x=x_range,
            y=y_kde,
            mode='lines',
            line=dict(width=2, color=colors[idx % len(colors)], dash='dash'),
            name=f'KDE f={frac:.2f}',
            showlegend=False,
            hovertemplate='Wealth: %{x:.2f}<br>Density: %{y:.3f}<extra></extra>'
        ),
        row=2, col=2
    )

# ========== Plot 4: Key Statistics vs Fraction ==========
# Mean Wealth
fig.add_trace(
    go.Scatter(
        x=df_results['fraction'],
        y=df_results['mean_wealth'],
        mode='lines',
        line=dict(width=2, color='#118AB2'),
        name='Mean Wealth',
        hovertemplate='Fraction: %{x:.3f}<br>Mean: %{y:.2f}<extra></extra>'
    ),
    row=3, col=1
)

# Median Wealth
fig.add_trace(
    go.Scatter(
        x=df_results['fraction'],
        y=df_results['median_wealth'],
        mode='lines',
        line=dict(width=2, color='#06D6A0'),
        name='Median Wealth',
        hovertemplate='Fraction: %{x:.3f}<br>Median: %{y:.2f}<extra></extra>'
    ),
    row=3, col=1
)

# Standard Deviation
fig.add_trace(
    go.Scatter(
        x=df_results['fraction'],
        y=df_results['std_wealth'],
        mode='lines',
        line=dict(width=2, color='#FFD166'),
        name='Std Dev',
        yaxis="y2",
        hovertemplate='Fraction: %{x:.3f}<br>Std Dev: %{y:.2f}<extra></extra>'
    ),
    row=3, col=1
)

# ========== Plot 5: Risk Measures ==========
# Probability of losing >50%
fig.add_trace(
    go.Scatter(
        x=df_results['fraction'],
        y=df_results['prob_ruin'],
        mode='lines+markers',
        line=dict(width=3, color='#EF476F'),
        marker=dict(size=6),
        name='P(Loss > 50%)',
        hovertemplate='Fraction: %{x:.3f}<br>P(Loss>50%): %{y:.2%}<extra></extra>'
    ),
    row=3, col=2
)

# Value at Risk (95%)
var_95 = []
for frac in df_results['fraction']:
    final_wealth = df_results.loc[df_results['fraction'] == frac, 'final_wealth_dist'].iloc[0]
    var = np.percentile(final_wealth, 5)  # 5th percentile
    var_95.append(var)

fig.add_trace(
    go.Scatter(
        x=df_results['fraction'],
        y=var_95,
        mode='lines+markers',
        line=dict(width=3, color='#7209B7', dash='dot'),
        marker=dict(size=6),
        name='VaR (95%)',
        hovertemplate='Fraction: %{x:.3f}<br>VaR 95%: %{y:.2f}<extra></extra>'
    ),
    row=3, col=2
)

# # ========== Plot 6: Sample Paths Comparison ==========
# # Create a small multiples plot for key fractions
# days = list(range(n_periods + 1))

# # Prepare data for small multiples
# for idx, frac in enumerate(key_fractions):
#     wealth_paths = wealth_samples[frac]
    
#     for i in range(min(5, len(wealth_paths))):
#         fig.add_trace(
#             go.Scatter(
#                 x=days,
#                 y=wealth_paths[i],
#                 mode='lines',
#                 line=dict(width=1.5, color=colors[idx % len(colors)]),
#                 opacity=0.7,
#                 name=f'f={frac:.2f}',
#                 legendgroup=f'group{idx}',
#                 showlegend=(i == 0),  # Only show legend for first path in group
#             ),
#             row=3, col=2
#         )

# ============================================================================
# 4. UPDATE LAYOUT AND STYLING
# ============================================================================

fig.update_layout(
    
    template='plotly_white',
    height=1200,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5,
        bgcolor='rgba(255, 255, 255, 0.8)'
    ),
    hovermode='x unified',
    plot_bgcolor='rgba(240, 242, 246, 0.5)',
    paper_bgcolor='white'
)

# Update axes
fig.update_xaxes(title_text="Time (Days)", row=1, col=1)
fig.update_yaxes(title_text="Wealth", tickprefix="", row=1, col=1)

fig.update_xaxes(title_text="Fraction in Risky Asset", row=2, col=1)
fig.update_yaxes(title_text="Expected Log Growth", row=2, col=1)

fig.update_xaxes(title_text="Final Wealth", range=[0, 5], row=2, col=2)
fig.update_yaxes(title_text="Probability Density", row=2, col=2)

fig.update_xaxes(title_text="Fraction in Risky Asset", row=3, col=1)
fig.update_yaxes(title_text="Wealth Statistics", row=3, col=1)
fig.update_yaxes(title_text="Std Dev", secondary_y=True, row=3, col=1)

fig.update_xaxes(title_text="Fraction in Risky Asset", row=3, col=2)
fig.update_yaxes(title_text="Probability / Wealth", row=3, col=2)

# Add annotations with summary
summary_text = f"""
<b>Summary Statistics</b><br>
Kelly Formula: {kelly_fraction:.3f}<br>
MC Optimal (log): {optimal_log_frac:.3f}<br>
MC Optimal (mean): {optimal_mean_frac:.3f}<br>
Simulations per fraction: {sim_per_fraction:,}<br>
Time periods: {n_periods}
"""

fig.add_annotation(
    text=summary_text,
    xref="paper", yref="paper",
    x=0.02, y=0.98,
    showarrow=False,
    align="left",
    bgcolor="rgba(255, 255, 255, 0.9)",
    bordercolor="#2E86AB",
    borderwidth=1,
    font=dict(size=11, family="Courier New, monospace")
)

# Add interactive buttons
fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            x=0.5,
            y=1.15,
            xanchor="center",
            showactive=True,
            buttons=list([
                dict(
                    label="Linear Scale",
                    method="update",
                    args=[{"visible": [True]*len(fig.data)}, 
                          {"yaxis.type": "linear", "yaxis2.type": "linear",
                           "yaxis3.type": "linear", "yaxis4.type": "linear",
                           "yaxis5.type": "linear", "yaxis6.type": "linear"}]
                ),
                dict(
                    label="Log Scale (Wealth)",
                    method="update",
                    args=[{"visible": [True]*len(fig.data)}, 
                          {"yaxis.type": "log", "yaxis2.type": "linear",
                           "yaxis3.type": "linear", "yaxis4.type": "log",
                           "yaxis5.type": "linear", "yaxis6.type": "log"}]
                )
            ]),
        ),
        dict(
            type="dropdown",
            direction="down",
            x=0.15,
            y=1.15,
            xanchor="left",
            showactive=True,
            buttons=list([
                dict(label="All Fractions",
                     method="update",
                     args=[{"visible": [True]*len(fig.data)}]),
                dict(label="Only ≤ Kelly",
                     method="update",
                     args=[{"visible": [(trace.x[0] <= kelly_fraction if hasattr(trace, 'x') and len(trace.x) > 0 else True) 
                                        for trace in fig.data]}]),
                dict(label="Only > Kelly",
                     method="update",
                     args=[{"visible": [(trace.x[0] > kelly_fraction if hasattr(trace, 'x') and len(trace.x) > 0 else True) 
                                        for trace in fig.data]}]),
            ]),
        )
    ]
)

# Show figure
fig.show()

# ============================================================================
# 5. ADDITIONAL VISUALIZATION: 3D SURFACE PLOT
# ============================================================================

# Create a 3D surface of mean vs volatility vs fraction
print("\nGenerating 3D surface plot...")

# Create grid of mu and vol values
mu_grid = np.linspace(0.02, 0.10, 20)
vol_grid = np.linspace(0.15, 0.40, 20)
optimal_fractions = np.zeros((len(mu_grid), len(vol_grid)))

# Compute Kelly optimal for each combination
for i, m in enumerate(mu_grid):
    for j, v in enumerate(vol_grid):
        optimal_fractions[i, j] = (m - rf) / (v**2)
        # Bound between 0 and 1
        optimal_fractions[i, j] = np.clip(optimal_fractions[i, j], 0, 1)

# Create 3D surface
fig_3d = go.Figure(data=[
    go.Surface(
        z=optimal_fractions,
        x=vol_grid,
        y=mu_grid,
        colorscale='Viridis',
        opacity=0.9,
        contours={
            "z": {"show": True, "start": 0, "end": 1, "size": 0.1}
        }
    )
])

fig_3d.update_layout(
    scene=dict(
        xaxis_title='Volatility (σ)',
        yaxis_title='Expected Return (μ)',
        zaxis_title='Optimal Fraction (f*)',
        zaxis_range=[0, 1],
        camera=dict(
            eye=dict(x=1.8, y=1.8, z=0.8)
        )
    ),
    height=600,
    margin=dict(l=0, r=0, b=0, t=40)
)

# Add annotations for current parameters
fig_3d.add_trace(
    go.Scatter3d(
        x=[vol],
        y=[mu],
        z=[kelly_fraction],
        mode='markers+text',
        marker=dict(size=8, color='red'),
        text=['Current'],
        textposition='top center'
    )
)

fig_3d.show()


Kelly optimal fraction: 0.222
Testing 41 fractions with 10,000 simulations each


/var/folders/fm/rwn89n9x1b94vk3rbr72zvtw0000gn/T/ipykernel_18085/532350127.py:69: RuntimeWarning:

divide by zero encountered in scalar divide




Monte Carlo Results:
Optimal for log growth: 0.350 (Kelly formula: 0.222)
Optimal for mean wealth: 0.950



Generating 3D surface plot...
